In [1]:
import os
import importlib
os.chdir('../app')

In [2]:
import preprocessing.preglobal as pg

In [3]:
import task_lib as tl

In [4]:
import fit_lib as fl

In [34]:
importlib.reload(fl)
importlib.reload(tl)

<module 'task_lib' from '/home/jovyan/MasterThesis/MasterThesis/analysis/task_lib.py'>

In [40]:
tl.execute_tasks() # RUN AFTER TASKS ARE ADDED TO SPAWN TASKS

python3 /home/jovyan/MasterThesis/MasterThesis/analysis/task_lib.py 616de0292d0d9900aee938db
python3 /home/jovyan/MasterThesis/MasterThesis/analysis/task_lib.py 616de0292d0d9900aee938dc
python3 /home/jovyan/MasterThesis/MasterThesis/analysis/task_lib.py 616de0292d0d9900aee938dd
python3 /home/jovyan/MasterThesis/MasterThesis/analysis/task_lib.py 616de0292d0d9900aee938de
python3 /home/jovyan/MasterThesis/MasterThesis/analysis/task_lib.py 616de0292d0d9900aee938df
python3 /home/jovyan/MasterThesis/MasterThesis/analysis/task_lib.py 616de0292d0d9900aee938e0
python3 /home/jovyan/MasterThesis/MasterThesis/analysis/task_lib.py 616de0292d0d9900aee938e1
python3 /home/jovyan/MasterThesis/MasterThesis/analysis/task_lib.py 616de0292d0d9900aee938e2
python3 /home/jovyan/MasterThesis/MasterThesis/analysis/task_lib.py 616de0292d0d9900aee938e3
python3 /home/jovyan/MasterThesis/MasterThesis/analysis/task_lib.py 616de0292d0d9900aee938e4
python3 /home/jovyan/MasterThesis/MasterThesis/analysis/task_lib.py 61

In [55]:
## MLE FITS - done

mle_fit_params = {
    'fit_book_notfixed':{
        'fitparameter':{
            'method':'mle',
            'book_convention':True,
            'fix_phi_dash':False
        }},
    'fit_book_fixed':{
        'fitparameter':{
            'method':'mle',
            'book_convention':True,
            'fix_phi_dash':True
        }},
    'fit_corrected_fixed': {
        'fitparameter':{
            'method':'mle',
            'book_convention':False,
            'fix_phi_dash':True
        }},
    'fit_corrected_notfixed': {
        'fitparameter':{
            'method':'mle',
            'book_convention':False,
            'fix_phi_dash':False
        }},
}

In [57]:
for i in pg.get_kn_entries({'selected':1}):
    task = {
    'task':'fit',
    'id':i['id'],
    'start':10.5*3600*1000,
    'stop':12.5*3600*1000
    }
    for k,v in mle_fit_params.items():
        taski = task.copy()
        taski['name'] = k
        for kk,vv in v['fitparameter'].items():
            taski[kk] = vv
        tl.add_task(taski)
    

In [ ]:
## COVARIANCE - done
for i in pg.get_kn_entries({'selected':1}):
    task = {
    'task':'covariance',
    'id':i['id'],
    'start':10.5*3600*1000,
    'stop':12.5*3600*1000
    }
    tl.add_task(task)
    

In [35]:
## MOMENT FITS - todo, after fit of moments is repaired

for i in pg.get_kn_entries({'selected':1}):
    task = {
    'id':i['id'],
    'task':'fit',
    'method':'moments',
    'start':10.5*3600*1000,
    'stop':12.5*3600*1000,
    'loadcachedcovariance':False,
    'powerfit':False,
    'lsqfit':True,
    'directcovar':True,
    'dt_range':[0.001,0.002,0.004]
}
    tl.add_task(task)

for i in pg.get_kn_entries({'selected':1}):
    task = {
    'id':i['id'],
    'task':'fit',
    'method':'moments',
    'start':10.5*3600*1000,
    'stop':12.5*3600*1000,
    'loadcachedcovariance':False,
    'powerfit':True,
    'lsqfit':False,
    'directcovar':True,
    'dt_range':[0.001,0.002,0.004]
}
    tl.add_task(task)

In [43]:
# CREATE SIMULATION FOR MOMENT OR MLE FIT RESULTS - IF NOT ALREADY SIMULATED

tbl = tl.dbconnect()
tasks = list(
        tbl.aggregate([{"$match":{"status":3,"error":None}},{"$sort":{"task.id":1}}])
    )

c = 0
for i in tasks:
    if 'method' not in i['task']:
        continue
    if len([t for t in tasks if 'origin' in t['task']and t['task']['task'] == 'simulate' and t['task']['origin']['id'] == i['_id']]) > 0:
        continue
    
    
    if i['task']['method'] == 'moments':
        for k,v in i['result']['g_params'].items():
            task = {'origin':{'id':i['_id'],'fit':k, 'task':i['task']}, 'task':'simulate', 'id':i['task']['id'], 'start':i['task']['start'], 'stop':i['task']['stop']}
            task['g_params'] = v
            print('nosimyet', task)
            c+=1
            #tl.add_task(task)
        
    if i['task']['method'] == 'mle':
        task = {'origin':{'id':i['_id'], 'task':i['task']}, 'task':'simulate', 'id':i['task']['id'], 'start':i['task']['start'], 'stop':i['task']['stop']}
        if 'phi_0' in i['result']:
            task['phi_0'] = i['result']['phi_0']
        task['g_params'] = i['result']['g_params']
        print('nosimyet', task)
        c+=1
        #tl.add_task(task)
print(c)

0


In [8]:
# EXECUTE SINGLE TASK DIRECTLY HERE FOR TESTING, E.G SIMULATION
result = tl.execute_task(
    {'origin': {'id': '123'}, 'task': 'simulate', 'id': '20150309_AAPL', 'start': 37800000.0, 'stop': 45000000.0, 'phi_0': 0.8976357559433028, 'g_params': [0.99, 7900.79560871683, 53.28371451939907]}
)
print(result)

mongo mongodb://192.168.0.94:27017/
redis 192.168.0.94
simulate using phi_dash 89.7635755943302
2975
5950
8925
11900
14875
{'sim_results': array([  2.49957072,   2.49960827,   2.49964919, ..., 317.85173948,
       317.8517422 , 317.85175701])}


In [9]:
# OR MOMENT FIT
res_moments = tl.execute_task( {
    'task':'fit',
    'method':'moments',
    'powerfit':True,
    'lsqfit':False,
    'loadcachedcovariance':True,
    'id':'20170912_AAPL',
    'start':10.5*3600*1000,
    'stop':12.5*3600*1000
    }

)

redis 192.168.0.94
start powerfit
start Yule Walker for powerfit using (c,omega,beta) [ 2.14781361 33.53665155  1.97202902]


/home/jovyan/MasterThesis/MasterThesis/analysis/fit_lib.py:102: RuntimeWarning: divide by zero encountered in true_divide
  f_power = lambda x,g,omega,beta: g*np.exp(omega)*beta/((1+np.exp(omega)*x)**(1+beta))


Fit ok
Fit ok
values for basinhopping, differential_evolution, difference 0.0006511421859404326 0.0006511424957664157 -3.098259831199948e-10


In [10]:
# OR MLE
res_mle = tl.add_task( {
    'task':'fit',
    'method':'mle',
    'book_convention':True,
    'fix_phi_dash':False,
    'id':'20170912_AAPL',
    'start':10.5*3600*1000,
    'stop':12.5*3600*1000
    }, execute_sync=False

)